In [3]:
import os
import csv
from tqdm import tqdm  # Importer tqdm pour le suivi de progression

# Fonction pour générer metadata.csv
def generate_metadata(input_dir, output_file):
    # Ouvrir le fichier CSV pour écrire
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter='|')
        writer.writerow(["audio_filename", "transcription"])  # Écrire l'en-tête
        
        # Compter le nombre total de fichiers .txt à traiter
        total_txt_files = sum([len(files) for root, dirs, files in os.walk(input_dir) if any(f.endswith('.txt') for f in files)])
        
        # Parcourir le dossier principal avec tqdm pour le suivi de progression
        for root, dirs, files in os.walk(input_dir):
            # Filtrer les fichiers .txt et .wav
            txt_files = [f for f in files if f.endswith('.txt')]
            wav_files = [f for f in files if f.endswith('.wav')]

            # Associer chaque fichier .wav à un fichier .txt
            for txt_file in tqdm(txt_files, desc="Processing files", total=total_txt_files, ncols=100):
                wav_file = txt_file.replace('.txt', '.wav')
                if wav_file in wav_files:
                    # Récupérer les chemins complets
                    txt_path = os.path.join(root, txt_file)
                    wav_path = os.path.join(root, wav_file)
                    
                    # Lire la transcription depuis le fichier .txt
                    with open(txt_path, 'r', encoding='utf-8') as f:
                        transcription = f.read().strip()
                    
                    # Écrire la ligne dans le fichier CSV
                    writer.writerow([wav_path, transcription])

    print(f"metadata.csv généré avec succès à l'emplacement {output_file}")



In [4]:
# Utilisation du script
input_dir = '/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/data_marte_wav/Clean-alignment'  # Le dossier de ton dataset
output_file = 'metadata.csv'  # Le fichier de sortie

# Générer le fichier metadata.csv
generate_metadata(input_dir, output_file)


Processing files:   0%|                                         | 25/11597 [00:00<01:00, 191.88it/s]

metadata.csv généré avec succès à l'emplacement metadata.csv


In [5]:
import pandas as pd

meta=pd.read_csv('metadata.csv')

In [6]:
meta.head()

,audio_filename|transcription
0,/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/data...
1,/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/data...
2,/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/data...
3,/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/data...
4,/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/data...


In [8]:
meta.shape[0]

5798

### formatter

In [9]:
import os

# Fonction pour formatter le dataset
def formatter(metadata_file, audio_dir):
    dataset = []

    with open(metadata_file, 'r') as file:
        next(file)  # Passer l'en-tête
        for line in file:
            # Séparer le chemin du fichier audio et la transcription
            audio_filename, transcript = line.strip().split('|')
            
            # Créer le chemin complet du fichier audio
            audio_path = os.path.join(audio_dir, audio_filename)
            
            # Ajouter à la liste sous forme de dictionnaire
            dataset.append({"audio_path": audio_path, "text": transcript})
    
    return dataset



In [10]:
# Utilisation du formatter
metadata_file = 'metadata.csv'  # Le fichier metadata.csv généré
audio_dir = ''  # Pas nécessaire ici car les chemins sont déjà complets dans le metadata.csv

# Formatter le dataset
formatted_data = formatter(metadata_file, audio_dir)

# Afficher les premières entrées du dataset formaté
for entry in formatted_data[:5]:
    print(entry)


{'audio_path': '/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/data_marte_wav/Clean-alignment/Clean/REV/REV_015/REV_015_005.wav', 'text': 'a mvus hala ndo mengayen ane tempel ememnyebe ayaaban a yob'}
{'audio_path': '/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/data_marte_wav/Clean-alignment/Clean/REV/REV_015/REV_015_006.wav', 'text': 'ndo bengles zamgbal ya bebele bindegele bengakui a tempel a nsen bebege efus eyie etoa mfuban efumu ane ten bebege bekanda be mvot a nkug'}
{'audio_path': '/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/data_marte_wav/Clean-alignment/Clean/REV/REV_015/REV_015_003.wav', 'text': 'beyiege dzia moses mboesye zamba ai dzia ntomba bo na mimboan mie mine anen mine adzo minkaman a nti zamba ngul mese mezen moe mene sosoo ai bebela a nkukuma meyon'}
{'audio_path': '/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/data_marte_wav/Clean-alignment/Clean/REV/REV_015/REV_015_007.wav', 'text': 'ndo tsit dzia ya tsit enyii yave bengles zamgbal kob mvot zamgbal etoa ndzalan ai olun zamba ya 

### splitter: train/dev/test

In [11]:
import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split

def split_dataset(metadata_file, output_dir, test_size=0.1, dev_size=0.1):
    # Charger le dataset
    df = pd.read_csv(metadata_file, sep="|", header=None, names=["audio_filename", "text"])
    
    # Séparer les données en train + dev et test
    train_dev, test = train_test_split(df, test_size=test_size, random_state=42)
    
    # Séparer train et dev
    train, dev = train_test_split(train_dev, test_size=dev_size / (1 - test_size), random_state=42)
    
    # Sauvegarder les fichiers
    os.makedirs(output_dir, exist_ok=True)
    train.to_csv(os.path.join(output_dir, "train.csv"), sep="|", index=False, header=False)
    dev.to_csv(os.path.join(output_dir, "dev.csv"), sep="|", index=False, header=False)
    test.to_csv(os.path.join(output_dir, "test.csv"), sep="|", index=False, header=False)
    
    print(f"✅ Dataset split completed:\n- Train: {len(train)} samples\n- Dev: {len(dev)} samples\n- Test: {len(test)} samples")

"""if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Diviser un dataset TTS en train, test et dev")
    parser.add_argument('--metadata_file', type=str, required=True, help="Chemin vers le fichier metadata.csv")
    parser.add_argument('--output_dir', type=str, required=True, help="Répertoire de sortie des fichiers divisés")
    parser.add_argument('--test_size', type=float, default=0.1, help="Proportion du dataset à utiliser pour le test (default: 10%)")
    parser.add_argument('--dev_size', type=float, default=0.1, help="Proportion du dataset à utiliser pour le dev (default: 10%)")

    args = parser.parse_args()
    
    split_dataset(args.metadata_file, args.output_dir, args.test_size, args.dev_size)
"""

'if __name__ == "__main__":\n    parser = argparse.ArgumentParser(description="Diviser un dataset TTS en train, test et dev")\n    parser.add_argument(\'--metadata_file\', type=str, required=True, help="Chemin vers le fichier metadata.csv")\n    parser.add_argument(\'--output_dir\', type=str, required=True, help="Répertoire de sortie des fichiers divisés")\n    parser.add_argument(\'--test_size\', type=float, default=0.1, help="Proportion du dataset à utiliser pour le test (default: 10%)")\n    parser.add_argument(\'--dev_size\', type=float, default=0.1, help="Proportion du dataset à utiliser pour le dev (default: 10%)")\n\n    args = parser.parse_args()\n    \n    split_dataset(args.metadata_file, args.output_dir, args.test_size, args.dev_size)\n'

In [14]:
output_dir= '/home/mendo/Downloads/LM/LM-5/LM-TTS-Main/tts'
split_dataset('metadata.csv',output_dir,test_size=0.1, dev_size=0.1)

✅ Dataset split completed:
- Train: 4639 samples
- Dev: 580 samples
- Test: 580 samples


In [15]:
import os
import argparse
import pandas as pd
from tqdm import tqdm

def get_book_name(audio_path):
    """Extrait le nom du livre à partir du chemin du fichier"""
    parts = audio_path.split(os.sep)
    return parts[-3]  # Le livre est dans le 3e niveau avant le fichier

def split_dataset_by_books(metadata_file, output_dir, test_books, dev_books):
    # Charger le dataset
    df = pd.read_csv(metadata_file, sep="|", header=None, names=["audio_filename", "text"])

    # Extraire le livre à partir du chemin
    tqdm.pandas(desc="Identification des livres")
    df["book"] = df["audio_filename"].progress_apply(get_book_name)

    # Filtrer les ensembles
    test_df = df[df["book"].isin(test_books)]
    dev_df = df[df["book"].isin(dev_books)]
    train_df = df[~df["book"].isin(test_books + dev_books)]  # Tout ce qui reste va dans train

    # Créer le répertoire de sortie
    os.makedirs(output_dir, exist_ok=True)

    # Sauvegarder les fichiers
    test_df[["audio_filename", "text"]].to_csv(os.path.join(output_dir, "test.csv"), sep="|", index=False, header=False)
    dev_df[["audio_filename", "text"]].to_csv(os.path.join(output_dir, "dev.csv"), sep="|", index=False, header=False)
    train_df[["audio_filename", "text"]].to_csv(os.path.join(output_dir, "train.csv"), sep="|", index=False, header=False)

    # Affichage des stats
    print(f"✅ Répartition terminée :")
    print(f"- Train : {len(train_df)} fichiers")
    print(f"- Dev   : {len(dev_df)} fichiers ({', '.join(dev_books)})")
    print(f"- Test  : {len(test_df)} fichiers ({', '.join(test_books)})")

"""if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Diviser un dataset TTS en train, test et dev selon les livres spécifiés")
    parser.add_argument('--metadata_file', type=str, required=True, help="Chemin vers le fichier metadata.csv")
    parser.add_argument('--output_dir', type=str, required=True, help="Répertoire de sortie des fichiers divisés")
    parser.add_argument('--test_books', nargs='+', required=True, help="Liste des livres pour le test (ex: JHN LUK)")
    parser.add_argument('--dev_books', nargs='+', required=True, help="Liste des livres pour le dev (ex: MRK ACT)")

    args = parser.parse_args()
    
    split_dataset_by_books(args.metadata_file, args.output_dir, args.test_books, args.dev_books)
"""

'if __name__ == "__main__":\n    parser = argparse.ArgumentParser(description="Diviser un dataset TTS en train, test et dev selon les livres spécifiés")\n    parser.add_argument(\'--metadata_file\', type=str, required=True, help="Chemin vers le fichier metadata.csv")\n    parser.add_argument(\'--output_dir\', type=str, required=True, help="Répertoire de sortie des fichiers divisés")\n    parser.add_argument(\'--test_books\', nargs=\'+\', required=True, help="Liste des livres pour le test (ex: JHN LUK)")\n    parser.add_argument(\'--dev_books\', nargs=\'+\', required=True, help="Liste des livres pour le dev (ex: MRK ACT)")\n\n    args = parser.parse_args()\n    \n    split_dataset_by_books(args.metadata_file, args.output_dir, args.test_books, args.dev_books)\n'

In [ ]:
get_book_name()